In [3]:
from tqdm import tqdm
import gym

from yaw_planner import Oxford


env = gym.make('gym-2d-perception-v0')
env.is_render = False
policy = Oxford(env.dt, env.dim)
# plt.ion()
max_step = 10000
rewards = []
steps = []
success = 0
fail = 0
for i in tqdm(range(max_step)):
    
    a = policy.plan(env.observation)
    observation, reward, done= env.step(a)
    if reward == 100:
        success += 1
    if reward == -100:
        fail += 1
    if done:
        env.reset()

    rewards.append(reward)
    steps.append(i)

    # env.render()

pygame 2.1.2 (SDL 2.0.16, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


TypeError: __init__() missing 1 required positional argument: 'params'

In [ ]:
print(success / (success + fail))

0.828125


In [3]:
from yaw_planner import LookAhead

env = gym.make('gym-2d-perception-v0')
env.render = False
policy = LookAhead(env.dt)
# policy = Oxford(env.dt, env.dim)
# plt.ion()
max_step = 10000
rewards = []
steps = []
success = 0
fail = 0
for i in tqdm(range(max_step)):
    
    a = policy.plan(env.observation)
    observation, reward, done= env.step(a)
    if reward == 100:
        success += 1
    if reward == -100:
        fail += 1
    if done:
        env.reset()

    rewards.append(reward)
    steps.append(i)

100%|██████████| 10000/10000 [03:32<00:00, 47.08it/s]


In [4]:
print(success / (success + fail))

0.8852459016393442


In [5]:
from yaw_planner import NoControl

env = gym.make('gym-2d-perception-v0')
env.render = False
policy = NoControl()
# policy = Oxford(env.dt, env.dim)
# plt.ion()
max_step = 10000
rewards = []
steps = []
success = 0
fail = 0
for i in tqdm(range(max_step)):
    
    a = policy.plan(env.observation)
    observation, reward, done= env.step(a)
    if reward == 100:
        success += 1
    if reward == -100:
        fail += 1
    if done:
        env.reset()

    rewards.append(reward)
    steps.append(i)

100%|██████████| 10000/10000 [03:38<00:00, 45.76it/s]


In [6]:
print(success / (success + fail))

0.6111111111111112
